In [34]:
import pandas as pd
df = pd.read_csv('union_pyth_clean_NOVA.csv')
df.shape

(881, 10)

Vymazání položek obsahující substringy typu škola, klub apod.

In [35]:
#SMAZÁNÍ všech položek obsahujicich ve sloupci NazevZarizeni substring "škol"
import re
search_pattern = re.compile(r'škol', flags=re.IGNORECASE)
rows_to_delete = df[df['NazevZarizeni'].apply(lambda x: bool(search_pattern.search(str(x))))].index
df = df.drop(rows_to_delete)

In [37]:
#SMAZÁNÍ všech položek obsahujicich ve sloupci Web substring "skol"
import re
search_pattern = re.compile(r'skol', flags=re.IGNORECASE)
rows_to_delete = df[df['Web'].apply(lambda x: bool(search_pattern.search(str(x))))].index
df = df.drop(rows_to_delete)

In [39]:
# Smazani radku obsahujiciho "Klub" ve sloupci NazevZarizeni
search_string = "Klub"
df = df[~df['NazevZarizeni'].apply(lambda x: search_string.lower() in str(x).lower())]


Vyhledani a smazani duplicit c. 1

In [41]:
#VYHLEDÁNÍ všech duplicit klin+neklin bez nazvu (keep=False zajistí zachování všech řádků) ve sloupcich NazevZarizeni PscZarizeni	UliceZarizeni	CisloDomovniOrientacni
dupl_found1 = df[df.duplicated(subset=['PscZarizeni', 'UliceZarizeni', 'CisloDomovniOrientacni'], keep=False) & df['UliceZarizeni'].notnull()].sort_values('CisloDomovniOrientacni')
dupl_found1.shape
#Toto jsou skutecne duplicity mezi obema datasety a budou vymazany nize

(318, 10)

In [42]:
#Specifikace (a ulozeni do promenne) radku ze souboru neklinickych (s nazvem), ktere jsou duplicitni s klinickymi, a proto budou smazany
to_drop_1 = df[df.duplicated(subset=['PscZarizeni', 'UliceZarizeni', 'CisloDomovniOrientacni'], keep=False) & df['UliceZarizeni'].notnull() & (df['ODBORNOST'] == "LogopedieF")]
#Pocet radku z neklin vymazanych jako duplicity klin/neklin
len(to_drop_1)

131

In [43]:
#Smazani duplicit klinicky/neklinicky 1., smazan radek neklinickych
df.drop(to_drop_1.index, inplace=True)

Vyhledani a smazani duplicit c. 2

In [45]:
#VYHLEDÁNÍ všech duplicit klin+neklin bez ulice s nazvem (keep=False zajistí zachování všech řádků)
dupl_found2 = df[df.duplicated(subset=['NazevZarizeni','PscZarizeni', 'CisloDomovniOrientacni'], keep=False)].sort_values('CisloDomovniOrientacni')
dupl_found2
#Toto jsou skutecne duplicity mezi obema datasety a budou vymazany nize

,NazevZarizeni,ObecZarizeni,PscZarizeni,UliceZarizeni,CisloDomovniOrientacni,Telefon,Web,LatZarizeni,LngZarizeni,ODBORNOST
36,Mgr. Petra Malíková,"Luže, Košumberk",53854,NaN,44,775130244,web neuveden,49.883465,16.039850,LogopedieF
740,Mgr. Petra Malíková,Luže,53854,NaN,44,+420775130244,web neuveden,49.883458,16.039847,logopedie


In [46]:
#Specifikace (a ulozeni do promenne) radku ze souboru neklinickych (s nazvem), ktere jsou duplicitni s klinickymi, a proto budou smazany
to_drop_2 = df[df.duplicated(subset=['NazevZarizeni', 'PscZarizeni', 'CisloDomovniOrientacni'], keep=False) & (df['ODBORNOST'] == "LogopedieF")]
#Pocet radku z neklin vymazanych jako duplicity klin/neklin
len(to_drop_2)

1

In [47]:
df.drop(to_drop_2.index, inplace=True)

Vyhledani a smazani duplicit c. 3

In [49]:
#VYHLEDÁNÍ všech duplicit klin+neklin
dupl_found3 = df[df.duplicated(subset=['NazevZarizeni', 'ObecZarizeni', 'UliceZarizeni', 'Telefon'], keep=False) & ~df['Telefon'].str.contains('telefon neuveden') & df['UliceZarizeni'].notnull()].sort_values('NazevZarizeni')
# liší se pouze v čísle domu, nutné jeden záznam smazat (Míša) - Budějovice č. 65 smazat, Vimperk  č. 268/14	smazat - podle pokynů Míši
dupl_found3

,NazevZarizeni,ObecZarizeni,PscZarizeni,UliceZarizeni,CisloDomovniOrientacni,Telefon,Web,LatZarizeni,LngZarizeni,ODBORNOST
572,Logopedická ambulance Centrum řeči s.r.o.,České Budějovice,37001,J. Š. Baara,1686/67,+420777302135,logopedie-ritterova.cz,48.979293,14.484338,logopedie
574,Logopedická ambulance Centrum řeči s.r.o.,České Budějovice,37001,J. Š. Baara,1685/65,+420777302135,logopedie-ritterova.cz,48.979310,14.484172,logopedie
272,Mgr. Elsa Císařová,Teplice,41501,Lípová,738,417977714,web neuveden,50.641602,13.836818,LogopedieF
280,Mgr. Elsa Císařová,Teplice,41501,Lípová,239/2,417977714,web neuveden,50.640995,13.835943,LogopedieF
391,PhDr. Jana Tláskalová,Vimperk,38501,1. máje,127/23,+420723442744,web neuveden,49.055923,13.780989,logopedie
780,PhDr. Jana Tláskalová,Vimperk,38501,1. máje,268/14,+420723442744,web neuveden,49.054836,13.779091,logopedie


In [50]:
#náhodně nalezena Elsa Císařová v LogopedieF, ale už je v logopedie, smazat
to_drop_3a = df[df['NazevZarizeni'] == 'Mgr. Elsa Císařová']
to_drop_3a

,NazevZarizeni,ObecZarizeni,PscZarizeni,UliceZarizeni,CisloDomovniOrientacni,Telefon,Web,LatZarizeni,LngZarizeni,ODBORNOST
272,Mgr. Elsa Císařová,Teplice,41501,Lípová,738,417977714,web neuveden,50.641602,13.836818,LogopedieF
280,Mgr. Elsa Císařová,Teplice,41501,Lípová,239/2,417977714,web neuveden,50.640995,13.835943,LogopedieF


In [51]:
#náhodně nalezena Elsa Císařová v LogopedieF, ale už je v logopedie, smazat
to_drop_3b = df[(df['NazevZarizeni'] == 'Logopedická ambulance Centrum řeči s.r.o.') &  (df['CisloDomovniOrientacni'] == '1685/65')]
to_drop_3b

,NazevZarizeni,ObecZarizeni,PscZarizeni,UliceZarizeni,CisloDomovniOrientacni,Telefon,Web,LatZarizeni,LngZarizeni,ODBORNOST
574,Logopedická ambulance Centrum řeči s.r.o.,České Budějovice,37001,J. Š. Baara,1685/65,+420777302135,logopedie-ritterova.cz,48.97931,14.484172,logopedie


In [52]:
to_drop_3c = df[(df['NazevZarizeni'] == 'PhDr. Jana Tláskalová') & (df['CisloDomovniOrientacni'] == '268/14')]
to_drop_3c

,NazevZarizeni,ObecZarizeni,PscZarizeni,UliceZarizeni,CisloDomovniOrientacni,Telefon,Web,LatZarizeni,LngZarizeni,ODBORNOST
780,PhDr. Jana Tláskalová,Vimperk,38501,1. máje,268/14,+420723442744,web neuveden,49.054836,13.779091,logopedie


In [53]:
# Smazaní 2x Elsa Císařová
df.drop(to_drop_3a.index, inplace=True)
# Smazaní Logopedická ambulance Centrum řeči s.r.o.
df.drop(to_drop_3b.index, inplace=True)
# Smazaní PhDr. Jana Tláskalová
df.drop(to_drop_3c.index, inplace=True)

In [54]:
df.shape

(732, 10)

Vyhledani a smazani duplicit c. 4

In [55]:
#Pro kontrolu zopakovani c. 1, ale vcetne ulic s nulovou hodnotou
dupl_found4 = df[df.duplicated(subset=['PscZarizeni', 'UliceZarizeni', 'CisloDomovniOrientacni'], keep=False)].sort_values('CisloDomovniOrientacni')
len(dupl_found4)
#nutne vyhodit duplicity s LogopedieF

81

In [ ]:
df.shape

In [56]:
#Specifikace (a ulozeni do promenne) radku ze souboru neklinickych (s nazvem), ktere jsou duplicitni s klinickymi, a proto budou smazany
to_drop_4 = df[df.duplicated(subset=['PscZarizeni', 'UliceZarizeni', 'CisloDomovniOrientacni'], keep=False) & (df['ODBORNOST'] == "LogopedieF")]
#Pocet radku z neklin vymazanych jako duplicity klin/neklin
len(to_drop_4)

3

In [57]:
df.drop(to_drop_4.index, inplace=True)

In [58]:
df.shape

(729, 10)

Vyhledani a vymazani nahodne nalezenych duplicit

In [59]:
dupl_found5 = df[df['ObecZarizeni'].str.contains('Třinec')]
dupl_found5
#nutné vymazat Logopro Máchova a Bezručova

,NazevZarizeni,ObecZarizeni,PscZarizeni,UliceZarizeni,CisloDomovniOrientacni,Telefon,Web,LatZarizeni,LngZarizeni,ODBORNOST
469,Mgr. Eliška Haltofová,Třinec,73961,Jablunkovská,408,telefon neuveden,web neuveden,49.669705,18.679984,logopedie
551,LOGOPRO s.r.o.,Třinec,73961,Máchova,1134,+420558998083,http://logopedietrinec.cz,49.670016,18.670179,logopedie
557,LOGOPRO s.r.o.,Třinec,73961,Bezručova,419,+420558998083,http://logopedietrinec.cz,49.674419,18.666929,logopedie
558,LOGOPRO s.r.o.,Třinec,73961,nám. Svobody,527,+420558998083,http://logopedietrinec.cz,49.669390,18.674915,logopedie
730,"Nemocnice Třinec, příspěvková organizace",Třinec,73961,Kaštanová,268,+420558309102,http://www.nemtr.cz,49.674753,18.690697,logopedie
846,Mgr. Zdenka Lakomá - CENTRUM LINIE,Třinec,73961,Kopernikova,623,telefon neuveden,web neuveden,49.668099,18.671842,logopedie


In [60]:
to_drop_5 = df[(df['NazevZarizeni'].str.contains('LOGOPRO')) & (df['UliceZarizeni'].str.contains('Bezručova|Máchova'))]
to_drop_5


,NazevZarizeni,ObecZarizeni,PscZarizeni,UliceZarizeni,CisloDomovniOrientacni,Telefon,Web,LatZarizeni,LngZarizeni,ODBORNOST
551,LOGOPRO s.r.o.,Třinec,73961,Máchova,1134,+420558998083,http://logopedietrinec.cz,49.670016,18.670179,logopedie
557,LOGOPRO s.r.o.,Třinec,73961,Bezručova,419,+420558998083,http://logopedietrinec.cz,49.674419,18.666929,logopedie


In [61]:
#Vymazání Logopro
df.drop(to_drop_5.index, inplace=True)

In [62]:
df.shape

(727, 10)

In [63]:
df[df['NazevZarizeni'].str.contains('Flower')]

,NazevZarizeni,ObecZarizeni,PscZarizeni,UliceZarizeni,CisloDomovniOrientacni,Telefon,Web,LatZarizeni,LngZarizeni,ODBORNOST
207,FlowerGarten,"Praha, Zbraslav",15600,Pod chaloupkami,1309,776637470,http://www.flowergarten.cz,49.965523,14.393744,LogopedieF


In [64]:
#Vymazání Flower
df = df[~df['NazevZarizeni'].str.contains('Flower')]

In [65]:
df.shape

(726, 10)

In [66]:
df.to_csv('union_pyth_VYCISTENY.csv', index=False)